In [1]:
!docker-compose up -d --build

time="2025-12-05T19:05:17+07:00" level=warning msg="d:\\CEDT\\Course\\Data_Science\\project\\docker-compose.yml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion"
 postgres Pulling 
 zookeeper Pulling 
 redis Pulling 
 kafka Pulling 
 6a229eaa0f8a Pulling fs layer 
 793133e9ba6b Pulling fs layer 
 4f4fb700ef54 Pulling fs layer 
 2d35ebdb57d9 Pulling fs layer 
 362eff98b603 Pulling fs layer 
 cffa735aaa17 Pulling fs layer 
 c84a6e003d26 Pulling fs layer 
 4f4fb700ef54 Already exists 
 21d3e41ee23f Pulling fs layer 
 702061a62ba9 Pulling fs layer 
 6188e3618e31 Pulling fs layer 
 d7999e2a96e3 Pulling fs layer 
 e24735bc229b Pulling fs layer 
 fff2675921f4 Pulling fs layer 
 e608323fc21f Pulling fs layer 
 c735131da4fc Pulling fs layer 
 c4ad802a5ffe Pulling fs layer 
 825cb6645075 Pulling fs layer 
 8b54b039ee30 Pulling fs layer 
 d0faedd0cc30 Pulling fs layer 
 7cad8264a6a6 Pulling fs layer 
 e1c7d925b76b Pulling fs layer 
 2e25d5ad6

In [ ]:
# โหดข้อมูลจากไฟล์ CSV เข้า Redis

import pandas as pd
import redis
import json

csv_file_path = 'bangkok_traffy.csv' 

redis_key = 'traffy_staging_queue'

r = redis.Redis(host='localhost', port=6379, db=0)

BATCH_SIZE = 10000

try:
    total_rows = 0
    pipe = r.pipeline()

    for chunk in pd.read_csv(csv_file_path, chunksize=BATCH_SIZE, dtype=str):
        
        chunk = chunk.fillna("")
        for row in chunk.itertuples(index=False):
            row_dict = row._asdict()
            json_data = json.dumps(row_dict, ensure_ascii=False)
            pipe.lpush(redis_key, json_data)
        
        pipe.execute()
        
        total_rows += len(chunk)
        print(f" ⏳ ส่งแล้ว {total_rows} แถว...", end='\r')

    print(f"✅ เสร็จสิ้น! ส่งทั้งหมด {total_rows} แถว")

except Exception as e:
    print(f"❌ Error: {e}")
